In [10]:
import numpy as np 
import pandas as pd
import mysql.connector as mysql
from datetime import date
from datetime import timedelta
import datetime
import os


In [11]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_leviscalendar =SQLdata.cursor()

mycursor_leviscalendar.execute("""SELECT YEAR,MONTH,MIN(DATE)start_date,MAX(DATE)end_date FROM dummy.levis_calendar \
GROUP BY 1,2 ORDER BY DATE ASC;""")

calendar_df = pd.DataFrame(mycursor_leviscalendar)
calendar_df.columns = mycursor_leviscalendar.column_names

calendar_df['start_date'] = pd.to_datetime(calendar_df['start_date'])
calendar_df['end_date'] = pd.to_datetime(calendar_df['end_date'])

# Get the current month start and end dates
current_date = pd.Timestamp.now()
current_month = current_date.month
current_year = current_date.year

In [12]:
#current_month_data =calendar_df[(calendar_df["YEAR"] == current_year) & (calendar_df["MONTH"] == current_date.strftime("%b"))]
#current_month_start_date = pd.to_datetime(current_month_data["start_date"].iloc[0], format='%Y-%m-%d').strftime('%Y-%m-%d')
#current_month_end_date = pd.to_datetime(current_month_data["end_date"].iloc[0], format='%Y-%m-%d').strftime('%Y-%m-%d')

# Get the last month start and end dates
if current_month == 1:
    last_month_data = calendar_df[(calendar_df["YEAR"] == current_year - 1) & (calendar_df["MONTH"] == "Dec")]
else:
    last_month_data = calendar_df[(calendar_df["YEAR"] == current_year) & (calendar_df["MONTH"] == current_date.replace(month=current_month - 1).strftime("%b"))]

last_month_start_date = pd.to_datetime(last_month_data["start_date"].iloc[0], format='%Y-%m-%d').strftime('%Y-%m-%d')
last_month_end_date = pd.to_datetime(last_month_data["end_date"].iloc[0], format='%Y-%m-%d').strftime('%Y-%m-%d')

#current_month_data_LY =calendar_df[(calendar_df["YEAR"] == current_year-1) & (calendar_df["MONTH"] == current_date.strftime("%b"))]
#current_month_start_date_LY = pd.to_datetime(current_month_data["start_date"].iloc[0], format='%Y-%m-%d').strftime('%Y-%m-%d')
#current_month_end_date_LY = pd.to_datetime(current_month_data["end_date"].iloc[0], format='%Y-%m-%d').strftime('%Y-%m-%d')

# Get the last month (Last Year) start and end dates
if current_month == 1:
    last_month_data_LY = calendar_df[(calendar_df["YEAR"] == current_year - 2) & (calendar_df["MONTH"] == "Dec")]
else:
    last_month_data_LY = calendar_df[(calendar_df["YEAR"] == current_year-1) & (calendar_df["MONTH"] == current_date.replace(month=current_month - 1).strftime("%b"))]

last_month_start_date_LY = pd.to_datetime(last_month_data["start_date"].iloc[0], format='%Y-%m-%d').strftime('%Y-%m-%d')
last_month_end_date_LY = pd.to_datetime(last_month_data["end_date"].iloc[0], format='%Y-%m-%d').strftime('%Y-%m-%d')

# assigning 'mmm-yy' to a variable
for index, row in last_month_data.iterrows():
    month_prefix = row['MONTH'][:3].upper()
    year_suffix = str(row['YEAR'])[-2:]
duration = f"{month_prefix}-{year_suffix}"

# assigning 'mmm-yy' to a variable (PY)
for index, row in last_month_data_LY.iterrows():
    month_prefix = row['MONTH'][:3].upper()
    year_suffix = str(row['YEAR'])[-2:]
duration_PY = f"{month_prefix}-{year_suffix}"

In [13]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_Winback =SQLdata.cursor()

#mycursor_Winback.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @last_month_start_date_LY;""") 
#mycursor_Winback.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @last_month_end_date_LY;""") 
#mycursor_Winback.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @last_month_start_date;""") 
#mycursor_Winback.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @last_month_end_date;""") 

## Importing 
## SKU report loyalty
mycursor_SkuDump =SQLdata.cursor()
mycursor_SkuDump.execute("""SELECT txnmappedmobile,modifiedtxndate,itemnetamount,dayssincelastvisit,modifiedbillno,
modifiedstorecode,frequencycount,itemqty FROM levis.sku_report_loyalty WHERE (modifiedtxndate \
BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""') AND \
itemqty>0;""")
skudump = pd.DataFrame(mycursor_SkuDump)
skudump.columns = mycursor_SkuDump.column_names

## Member report
mycursor_memberreport =SQLdata.cursor()

#mycursor_memberreport.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @c;""") 
#mycursor_memberreport.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @d;""") 
#mycursor_memberreport.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @g;""") 
#mycursor_memberreport.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @h;""") 

mycursor_memberreport.execute("""SELECT mobile,gender,enrolledstorecode,date(dateofbirth)dateofbirth,email FROM \
levis.member_report WHERE modifiedenrolledon BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""';""")

memberdump = pd.DataFrame(mycursor_memberreport)
memberdump.columns = mycursor_memberreport.column_names

## Sku report NonLoyalty
mycursor_NLSkuDump =SQLdata.cursor()

#mycursor_NLSkuDump.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @c;""") 
#mycursor_NLSkuDump.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @d;""") 
#mycursor_NLSkuDump.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @g;""") 
#mycursor_NLSkuDump.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @h;""") 

mycursor_NLSkuDump =SQLdata.cursor()
mycursor_NLSkuDump.execute("""SELECT modifiedtxndate,modifiedbillno,modifiedstorecode,itemnetamount 
FROM levis.sku_report_nonloyalty WHERE modifiedtxndate BETWEEN '"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""' \
AND itemqty>0;""")
NLskudump = pd.DataFrame(mycursor_NLSkuDump)
NLskudump.columns = mycursor_NLSkuDump.column_names

## Store master
mycursor_storemaster =SQLdata.cursor()

mycursor_storemaster.execute("""SELECT storecode,region FROM levis.store_master group by 1;""")

storedump = pd.DataFrame(mycursor_storemaster)
storedump.columns = mycursor_storemaster.column_names

## Winback
skudump['tag'] = None
skudump.loc[(skudump['dayssincelastvisit'] >= 731) & (skudump['dayssincelastvisit'] <= 820), 'tag'] = '731-820'
skudump.loc[(skudump['dayssincelastvisit'] >= 821) & (skudump['dayssincelastvisit'] <= 910), 'tag'] = '821-910'
skudump.loc[(skudump['dayssincelastvisit'] >= 911) & (skudump['dayssincelastvisit'] <= 1000), 'tag'] = '911-1000'
skudump.loc[(skudump['dayssincelastvisit'] >= 1001) & (skudump['dayssincelastvisit'] <= 1200), 'tag'] = '1001-1200'
skudump.loc[(skudump['dayssincelastvisit'] >= 1201) & (skudump['dayssincelastvisit'] <= 1500), 'tag'] = '1201-1500'
skudump.loc[(skudump['dayssincelastvisit'] >= 1501) & (skudump['dayssincelastvisit'] <= 2000), 'tag'] = '1501-2000'
skudump.loc[skudump['dayssincelastvisit'] > 2000, 'tag'] = '2000+'

winback = skudump.groupby('tag')['txnmappedmobile'].nunique().reset_index()
winback.columns = ['tag', 'Customers']

winback_transpose = winback.set_index('tag').transpose().sort_index(ascending=False)
winback_transpose['Duration'] = duration
columns = ['Duration'] + [col for col in winback_transpose if col != 'Duration']
winback_output = winback_transpose[columns]

# Enrollments
enrollments = memberdump.shape[0]

female_enrolments = memberdump[(memberdump['gender'].str.contains('FEMALE|female', case=False, na=False)) & 
                                (memberdump['gender'].notna())]['mobile'].nunique()
                               
male_enrolments = memberdump[(memberdump['gender'].str.contains('Male|male', case=False, na=False)) & 
                              (memberdump['gender'].notna())]['mobile'].nunique()

unmapped_enrolments = memberdump[(memberdump['gender'].isna()) | (memberdump['gender'] == '')]['mobile'].nunique()

dob = memberdump[memberdump['dateofbirth'].notna()]['mobile'].nunique()

email = memberdump[memberdump['email'].notna()]['mobile'].nunique()

enrollments_op = pd.DataFrame({
    'enrollments': [enrollments],
    'female_enrolments': [female_enrolments],
    'male_enrolments': [male_enrolments],
    'unmapped_enrolments': [unmapped_enrolments],
    'dob': [dob],
    'email': [email]
})

enrollments_op['Duration'] = duration
columns = ['Duration'] + [col for col in enrollments_op if col != 'Duration']
enrollments_output = enrollments_op[columns]


# MOM metrics calculation
loyalty_customers = skudump['txnmappedmobile'].nunique()
repeaters = skudump.loc[skudump['frequencycount'] > 1, 'txnmappedmobile'].nunique()
winback_customers = skudump.loc[skudump['dayssincelastvisit'] > 730, 'txnmappedmobile'].nunique()

nonloyalty_txn = NLskudump[['modifiedtxndate', 'modifiedbillno', 'modifiedstorecode']].drop_duplicates().shape[0]
nonloyalty_sales = NLskudump['itemnetamount'].sum()

MOM_op = pd.DataFrame({
    'customers': [loyalty_customers],
    'repeaters': [repeaters],
    'WinbackCustomers': [winback_customers],
    'nonloyalty_txn': [nonloyalty_txn],
    'Nonloyaltysales': [nonloyalty_sales]})

MOM_op['Duration'] = duration
columns = ['Duration'] + [col for col in MOM_op if col != 'Duration']
MOM_output = MOM_op[columns]


In [14]:
## Perform the SQL-like operations using Pandas
# memberdump
# storedump
# skudump
# NLskudump

a = memberdump.merge(storedump, left_on='enrolledstorecode', right_on='storecode', how='left').groupby('region').apply(lambda x: pd.Series({
    'NewEnrollment': x['mobile'].nunique(),
    'WomenEnrolments': (x['gender'] == 'Female').sum(),
    'MenEnrolments': (x['gender'] == 'Male').sum()
}))

b = skudump.merge(storedump, left_on='modifiedstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'RedTabSales': x['itemnetamount'].sum(),
    'TotalTransactingCustomers': x['txnmappedmobile'].nunique(),
    'Transacters_1to365D': x[(x['dayssincelastvisit'] >= 1) & (x['dayssincelastvisit'] <= 365)]['txnmappedmobile'].nunique(),
    'Transacters_366to730D': x[(x['dayssincelastvisit'] >= 366) & (x['dayssincelastvisit'] <= 730)]['txnmappedmobile'].nunique(),
    'Winback': x[x['dayssincelastvisit'] > 730]['txnmappedmobile'].nunique(),
    'RedTabTxns': x['modifiedbillno'].nunique()
}))

c = NLskudump.merge(storedump, left_on='modifiedstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'NLTxns': x['modifiedbillno'].nunique()
}))

enrolledtransacted = pd.merge(memberdump, skudump, left_on='mobile', right_on='txnmappedmobile', how='inner')[['mobile','enrolledstorecode']]

d = enrolledtransacted.merge(storedump, left_on='enrolledstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'NewlyEnrolledTransacted': x['mobile'].nunique()
}))

region_level = pd.concat([a, b, c, d], axis=1)
region_level['TotalTxns'] = region_level['RedTabTxns'] + region_level['NLTxns']

region_level['Duration'] = duration
columns = ['Duration'] + [col for col in region_level if col != 'Duration']
region_level_output = region_level[columns]

In [15]:
#### Importing Tables for Region wise metrics calculations (for previous year)

## Sku report Loyalty
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")

mycursor_SkuDump =SQLdata.cursor()

#mycursor_SkuDump.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @c;""") 
#mycursor_SkuDump.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @d;""") 

mycursor_SkuDump =SQLdata.cursor()
mycursor_SkuDump.execute("""SELECT txnmappedmobile,modifiedtxndate,itemnetamount,dayssincelastvisit,modifiedbillno,
modifiedstorecode,frequencycount,itemqty FROM levis.sku_report_loyalty WHERE (modifiedtxndate \
BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""') AND \
itemqty>0;""")
PYskudump = pd.DataFrame(mycursor_SkuDump)
PYskudump.columns = mycursor_SkuDump.column_names

## Sku report NonLoyalty
mycursor_NLSkuDump =SQLdata.cursor()

#mycursor_NLSkuDump.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @c;""") 
#mycursor_NLSkuDump.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @d;""") 

mycursor_NLSkuDump =SQLdata.cursor()
mycursor_NLSkuDump.execute("""SELECT modifiedtxndate,modifiedbillno,modifiedstorecode 
FROM levis.sku_report_nonloyalty WHERE (modifiedtxndate \
BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""') AND itemqty>0;""")
PYNLskudump = pd.DataFrame(mycursor_NLSkuDump)
PYNLskudump.columns = mycursor_NLSkuDump.column_names

## Member report
mycursor_memberreport =SQLdata.cursor()

#mycursor_memberreport.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @c;""") 
#mycursor_memberreport.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @d;""") 

mycursor_memberreport.execute("""SELECT mobile,gender,enrolledstorecode FROM levis.member_report WHERE modifiedenrolledon \
BETWEEN '"""+last_month_start_date_LY+"""' AND '"""+last_month_end_date_LY+"""';""")

PYmemberdump = pd.DataFrame(mycursor_memberreport)
PYmemberdump.columns = mycursor_memberreport.column_names

In [16]:
## Perform sql like calculations and aggregations with python functions 

a = PYmemberdump.merge(storedump, left_on='enrolledstorecode', right_on='storecode', how='left').groupby('region').apply(lambda x: pd.Series({
    'NewEnrollment': x['mobile'].nunique(),
    'WomenEnrolments': (x['gender'] == 'Female').sum(),
    'MenEnrolments': (x['gender'] == 'Male').sum()
}))

b = PYskudump.merge(storedump, left_on='modifiedstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'RedTabSales': x['itemnetamount'].sum(),
    'TotalTransactingCustomers': x['txnmappedmobile'].nunique(),
    'Transacters_1to365D': x[(x['dayssincelastvisit'] >= 1) & (x['dayssincelastvisit'] <= 365)]['txnmappedmobile'].nunique(),
    'Transacters_366to730D': x[(x['dayssincelastvisit'] >= 366) & (x['dayssincelastvisit'] <= 730)]['txnmappedmobile'].nunique(),
    'Winback': x[x['dayssincelastvisit'] > 730]['txnmappedmobile'].nunique(),
    'RedTabTxns': x['modifiedbillno'].nunique()
}))

c = PYNLskudump.merge(storedump, left_on='modifiedstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'NLTxns': x['modifiedbillno'].nunique()
}))

PYenrolledtransacted = pd.merge(memberdump, skudump, left_on='mobile', right_on='txnmappedmobile', how='inner')[['mobile','enrolledstorecode']]

d = PYenrolledtransacted.merge(storedump, left_on='enrolledstorecode', right_on='storecode', how='left') \
    .groupby('region').apply(lambda x: pd.Series({
    'NewlyEnrolledTransacted': x['mobile'].nunique()
}))

PY_region_level = pd.concat([a, b, c, d], axis=1)
PY_region_level['TotalTxns'] = PY_region_level['RedTabTxns'] + PY_region_level['NLTxns']

PY_region_level['Duration'] = duration_PY
columns = ['Duration'] + [col for col in PY_region_level if col != 'Duration']
PY_region_level_output = PY_region_level[columns]

In [ ]:
## Gender tag
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_Gendertag =SQLdata.cursor()

#mycursor_Gendertag.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @c;""") 
#mycursor_Gendertag.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @d;""")
#mycursor_Gendertag.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @g;""") 
#mycursor_Gendertag.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @h;""")

mycursor_Gendertag.execute("""SELECT txnmappedmobile,itemqty,uniqueitemcode,modifiedtxndate FROM levis.sku_report_loyalty \
WHERE txnmappedmobile in \
(select txnmappedmobile from levis.sku_report_loyalty where (modifiedtxndate between \
'"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""') and itemqty>0) \
and modifiedtxndate<='"""+last_month_end_date+"""' AND itemqty>0;""")

GenderDump = pd.DataFrame(mycursor_Gendertag)
GenderDump.columns = mycursor_Gendertag.column_names

mycursor_Gendertag.execute("""SELECT uniqueitemcode,gender FROM levis.item_master;""")

itemmaster = pd.DataFrame(mycursor_Gendertag)
itemmaster.columns = mycursor_Gendertag.column_names

GenderDumpUC = pd.merge(GenderDump, itemmaster, on='uniqueitemcode', how='left')[['txnmappedmobile','itemqty','uniqueitemcode',
                                                                   'modifiedtxndate','gender']]

## Monthly memberdump
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_monthlymember =SQLdata.cursor()

#mycursor_monthlymember.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @c;""") 
#mycursor_monthlymember.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2023' INTO @d;""")
#mycursor_monthlymember.execute("""SELECT MIN(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @g;""") 
#mycursor_monthlymember.execute("""SELECT MAX(DATE) FROM dummy.levis_calendar WHERE MONTH='Mar' AND YEAR='2024' INTO @h;""")

mycursor_monthlymember.execute("""SELECT mobile,gender FROM levis.member_report where mobile in \
(SELECT DISTINCT txnmappedmobile FROM levis.sku_report_loyalty where(modifiedtxndate between \
''"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""') and itemqty>0) \
group by 1;""")

monthlymember = pd.DataFrame(mycursor_monthlymember)
monthlymember.columns = mycursor_monthlymember.column_names

In [24]:
## Apply the conditions and perform the aggregations
## Python never shows null and blank values count by normal code, check below code for null and blank counts
#GenderDumpUC[GenderDumpUC['gender'].isnull()] #to see null values in gender column
#GenderDumpUC[GenderDumpUC['gender'] == ''] #to see blank values in gender column
GenderDumpUC['gender'].fillna('notag', inplace=True)

women_condition = GenderDumpUC['gender'].str.contains('WOMENS|Women|Total Womens', case=False)
men_condition = (~women_condition) & GenderDumpUC['gender'].str.contains('Mens|Men|Total Mens', case=False)

result = GenderDumpUC.groupby('txnmappedmobile').agg(
    womenqty=('itemqty', lambda x: x[women_condition].sum()),
    menqty=('itemqty', lambda x: x[men_condition].sum()),
    totalqty=('itemqty', 'sum')
).reset_index()

In [70]:
## Updating percentages
result['womenper'] = result['womenqty'] / result['totalqty'].replace(0, pd.NA)
result['menper'] = result['menqty'] / result['totalqty'].replace(0, pd.NA)

## Updating customer gender by levis.member_report DataFrame
merged_df = pd.merge(result, monthlymember, left_on='txnmappedmobile', right_on='mobile', how='left')
result['gender'] = merged_df['gender']

## Updating dslv from skudump
skudump_unique_dslv = skudump.groupby('txnmappedmobile')['dayssincelastvisit'].min().reset_index()

dslv_merger = pd.merge(result, skudump_unique_dslv, on='txnmappedmobile', how='left')
result['dayssincelastvisit'] = dslv_merger['dayssincelastvisit']


In [79]:
## Tagging final gender tags
result.loc[result['gender'].str.contains('FEMALE|female', case=False, na=False), 'seg'] = 'p2'
result.loc[(result['gender'].str.contains('MALE|male', case=False, na=False)) & (result['seg'].isnull()), 'seg'] = 'p1'

result.loc[(result['menper'] > 0.7) & (result['seg'].isnull()), 'seg'] = 'p3'
result.loc[(result['womenper'] > 0.7) & (result['seg'].isnull()), 'seg'] = 'p4'
result.loc[result['seg'].isnull(), 'seg'] = 'others'

#result.seg.value_counts()

In [177]:
## Define the conditions and corresponding labels for the segmentation
result_dslv_notnull = result[result['dayssincelastvisit'].notnull()]
result_dslv_notnull['dayssincelastvisit'] = result_dslv_notnull['dayssincelastvisit'].astype('int32')

conditions = [
    (result_dslv_notnull['dayssincelastvisit'].between(1, 730, inclusive='both')),
    (result_dslv_notnull['dayssincelastvisit'] > 730),
    (result_dslv_notnull['dayssincelastvisit'] < 1)
]
tags = ['ActiveCustomer','LapsedCustomer','NewCustomer']

result_dslv_notnull['lapsation'] = None

## Update the 'lapsation' column based on conditions
for condition, lapsation in zip(conditions, tags):
    result_dslv_notnull.loc[condition, 'lapsation'] = lapsation

## Output
gender_output = result_dslv_notnull.groupby(['seg', 'lapsation']).agg({'txnmappedmobile': 'nunique'}).reset_index()


## QC
#result_dslv_notnull[result_dslv_notnull['lapsation']=='NewCustomer']['dayssincelastvisit'].min()

# Group by 'seg' and 'Lapsation', and count the number of distinct customers
#result = result.groupby(['seg', 'Lapsation']).agg({'txnmappedmobile': 'nunique'}).reset_index()
#result.columns = ['FinalGender', 'Lapsation', 'Cust']
#result.drop(['Lapsation'],axis=1, inplace=True)

C:\Users\pankajkumar\AppData\Local\Temp\ipykernel_32500\816479876.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_dslv_notnull['dayssincelastvisit'] = result_dslv_notnull['dayssincelastvisit'].astype('int32')
C:\Users\pankajkumar\AppData\Local\Temp\ipykernel_32500\816479876.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_dslv_notnull['lapsation'] = None


In [ ]:
## Lifecycle split
last_month_LC = "dummy.Lifecycle_LevisIn_25Feb24"
current_month_LC = "dummy.Lifecycle_LevisIn_31Mar24"

In [ ]:
SQLdata = mysql.connect(host = '20.105.3.14',user = 'PankajKumar',passwd = 'K^mar@32$',database = "levis")
mycursor_Gendertag =SQLdata.cursor()

mycursor_Gendertag.execute("""SELECT txnmappedmobile,itemqty,uniqueitemcode,modifiedtxndate FROM levis.sku_report_loyalty \
WHERE txnmappedmobile in \
(select txnmappedmobile from levis.sku_report_loyalty where (modifiedtxndate between \
'"""+last_month_start_date+"""' AND '"""+last_month_end_date+"""') and itemqty>0) \
and modifiedtxndate<='"""+last_month_end_date+"""' AND itemqty>0;""")